<a href="https://colab.research.google.com/github/khokhakhokha/chatbot-for-business-website/blob/main/rag_chatbot_on_custom_website_content.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Import

In [ ]:
!pip install langchain
!pip install langchain-community
!pip install langchain-google-genai
!pip install langchain-openai
!pip install  langchain-chroma
!pip install chromadb
!pip install huggingface_hub
!pip install streamlit
!pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.6/449.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.77
    Uninstalling langchain-core-0.3.77:
      Successfully uninstalled langchain-core-0.3.77
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.3 MB

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.4/128.4 kB 10.2 MB/s eta

In [ ]:
from pathlib import Path
from google.colab import drive
import os, tempfile, glob, random
# documents loaders
from langchain_community.document_loaders import DirectoryLoader, TextLoader
# text splitters
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
# Chroma: vectorstore
from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma
# # LLM: openai and google_genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings


In [ ]:
# Mount Google Drive
drive.mount('/content/drive')from langchain.schema import BaseOutputParser
DEFAULT_TEMPLATE= """
### Instruction: Youre name is Albert, you're a customer support agent from a Start up called Exclusible, that is talking to a customer. Use only the chat history and the information in this parenthesis (
The CEO of Exclusible is Thibault Launay.
Every brand name after the url https://www.exclusible.com/, like this one: https://www.exclusible.com/amedeo are projects created by Exclusible.
Exclusible has work with buggati, alpine, hugo boss, ...
 {context}

Your startup sells 4 services:
- NFT creation,
- Metaverse,
- Marketing Agency
- Immersive Real Estate
), to answer in a helpful manner to the question. If you don't know the answer - say contact contact@exclusible.com for more information.
Keep your replies short, compassionate and informative.
{chat_history}
### Input: {question}
### Response:
""".strip()

class ChatBot:
    def __init__(
        self,
        text_pipeline: HuggingFacePipeline,
        embeddings: HuggingFaceEmbeddings,
        training_data: list,
        prompt_template: str = DEFAULT_TEMPLATE,
        verbose: bool = False
    ):
        prompt = PromptTemplate(
            input_variables=["context", "question", "chat_history"], template=prompt_template, output_parser=CleanupOutputParser())
        self.chain = self._create_chain(text_pipeline, prompt, verbose)
        self.db = self._embed_data(training_data, embeddings)

    def _create_chain(
        self,
        text_pipeline: HuggingFacePipeline,
        prompt: PromptTemplate,
        verbose: bool = False
    ):
        memory = ConversationBufferMemory(
            memory_key="chat_history",
            human_prefix="### Input",
            ai_prefix="### Response",
            input_key="question",
            output_key="output_text",
            return_messages=False
        )
        return load_qa_chain(text_pipeline, chain_type="stuff", prompt=prompt, memory=memory, verbose=verbose)

    def _embed_data(self, training_data, embeddings: HuggingFaceEmbeddings) -> Chroma:
        urls = training_data.urls
        pdfs = training_data.pdfs
        online_pdfs = training_data.online_pdfs
        documents_dir = training_data.documents_dir

        documents_loader = []
        if (len(documents_dir) > 0):
            for document_dir in documents_dir:
                documents_loader.append(DirectoryLoader(
                    document_dir, glob="**/*txt"))

        urls_loader = []
        if (len(urls) > 1):
            urls_loader.append(UnstructuredURLLoader(urls=urls))

        pdfs_loader = []
        if (len(pdfs) > 0):
            for pdf in pdfs:
                pdfs_loader.append(PyPDFLoader(pdf))

        online_pdfs_loader = []
        if (len(online_pdfs) > 0):
            for online_pdf in online_pdfs:
                online_pdfs_loader.append(PyPDFLoader(online_pdf))

        loaders = [urls_loader, pdfs_loader,
                   online_pdfs_loader, documents_loader]

        docs = []
        for loader in loaders:
            if (len(loader) > 0):
                docs.extend(loader[0].load())
        #print('docs',docs)
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=512, chunk_overlap=40)
        texts = text_splitter.split_documents(docs)
        return Chroma.from_documents(texts, embeddings)

    def __call__(self, user_input: str) -> str:
        docs = self.db.similarity_search(user_input)
        return self.chain.run({"input_documents": docs, "question": user_input})



# Define your base folder in Drive (you can change 'MyDrive' path if needed)
BASE_DIR = Path("/content/drive/MyDrive/data")
# Create the directories if they don't exist
(BASE_DIR / "tmp").mkdir(parents=True, exist_ok=True)
(BASE_DIR / "vector_stores").mkdir(parents=True, exist_ok=True)
# Define paths
TMP_DIR = BASE_DIR / "tmp"
LOCAL_VECTOR_STORE_DIR = BASE_DIR / "vector_stores"

Mounted at /content/drive


##API KEYS

In [ ]:
google_api_key =  os.environ.get("GOOGLE_API_KEY")
HF_key = os.environ.get("HUGGINGFACEHUB_API_TOKEN")

##Document Loaders

In [ ]:
def langchain_document_loader(TMP_DIR):


    documents = []

    txt_loader = DirectoryLoader(
        TMP_DIR.as_posix(), glob="**/*.txt", loader_cls=TextLoader, show_progress=True
    )
    documents.extend(txt_loader.load())

    return documents

In [ ]:
# load documents
documents = langchain_document_loader(TMP_DIR)
print(f"\nNumber of documents: {len(documents)}")

100%|██████████| 5/5 [00:01<00:00,  2.81it/s]


Number of documents: 5


##Text splitters


In [ ]:
# Create a RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n", "\n", " ", ""],
    chunk_size = 1600,
    chunk_overlap= 200
)

##Vectorsores and Embeddings




###Embeddings

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(
            model="models/embedding-001",
            google_api_key=google_api_key
        )

###Vectorestores

In [ ]:
def create_vectorstore(embeddings, documents, vectorstore_name):
    """Create and persist a Chroma vector database."""

    # Define persistence directory
    persist_directory = Path("./data/vector_stores") / vectorstore_name
    persist_directory.mkdir(parents=True, exist_ok=True)

    # Create Chroma vector store
    vector_store = Chroma.from_documents(
        documents=documents,
        embedding=embeddings,
        persist_directory=str(persist_directory)
    )

    # Persist data to disk
    vector_store.persist()


    return vector_store, str(persist_directory)

###Load chroma vectorestore

In [ ]:
vector_store = Chroma(
    persist_directory = LOCAL_VECTOR_STORE_DIR.as_posix() + "/Vit_All_Google_Embeddings",
    embedding_function=embeddings)



## Retrivers

In [ ]:
def vectorestore_retriver():
  return vector_store.as_retriever()


## Retrival : all the block

In [ ]:
def setup_retrieval_pipeline(
    create_new_vectorstore=True,
    vectorstore_name="Vit_All_Google_Embeddings",
    chunk_size=1600,
    chunk_overlap=200,
    google_api_key="***",
):
    """
    🔍 Sets up the retrieval pipeline:
    Loads documents → splits into chunks → embeds them → creates or loads a Chroma vectorstore → builds a retriever.

    Parameters:
        create_new_vectorstore (bool):
            If True, creates a new Chroma vectorstore from source documents.
            If False, loads an existing vectorstore from disk.

        vectorstore_name (str):
            Directory name (inside LOCAL_VECTOR_STORE_DIR) where the Chroma vectorstore is stored.

        chunk_size (int):
            Maximum number of characters per text chunk when splitting documents.

        chunk_overlap (int):
            Number of overlapping characters between consecutive chunks to preserve context.

        google_api_key (str):
            Google API key used to initialize Google Generative AI embeddings.

    Returns:
        retriever (BaseRetriever):
            A configured retriever ready to query the Chroma vectorstore.
    """

    try:
        # --- Initialize Embeddings ---
        print("🧠 Initializing Google Generative AI Embeddings ...")
        embeddings = GoogleGenerativeAIEmbeddings(
            model="models/embedding-001",
            google_api_key=google_api_key
        )

        # --- Create or Load Vectorstore ---
        if create_new_vectorstore:
            print(f"🔧 Creating new Chroma vectorstore: {vectorstore_name} ...")

            # 1️⃣ Load documents
            documents = langchain_document_loader(TMP_DIR)

            # 2️⃣ Split into chunks
            splitter = RecursiveCharacterTextSplitter(
                separators=["\n\n", "\n", " ", ""],
                chunk_size=chunk_size,
                chunk_overlap=chunk_overlap
            )
            chunks = splitter.split_documents(documents)

            # 3️⃣ Create and persist vectorstore
            vector_store = create_vectorstore(
                embeddings=embeddings,
                documents=chunks,
                vectorstore_name=vectorstore_name,
            )

        else:
            print(f"📦 Loading existing Chroma vectorstore: {vectorstore_name} ...")
            vector_store = Chroma(
                persist_directory=(LOCAL_VECTOR_STORE_DIR / vectorstore_name).as_posix(),
                embedding_function=embeddings
            )

        # --- Build Retriever ---
        retriever = vectorestore_retriver()

        # --- Summary Output ---
        chunk_count = vector_store._collection.count()
        print(f"\n✅ Retrieval pipeline ready!")
        print(f"   ➤ Vectorstore: {vectorstore_name}")
        print(f"   ➤ Embeddings: Google Generative AI")
        print(f"   ➤ Chunks available: {chunk_count}")

        return retriever

    except Exception as e:
        print(f"⚠️ Error in setup_retrieval_pipeline: {e}")
        raise


## Chatmodel

In [ ]:
def instantiate_llm(api_key,  temperature=0.5, top_p=0.95):
    """
    🧠 Instantiate a Google Generative AI (Gemini) LLM in LangChain.

    Parameters:
        api_key (str):
            Your Google API key used to authenticate with the Gemini model.
        temperature (float):
            Controls randomness in responses. Range: 0.0–1.0 (default=0.5).
        top_p (float):
            Nucleus sampling parameter. Range: 0.0–1.0 (default=0.95).

    Returns:
        ChatGoogleGenerativeAI:
            A configured Google Generative AI LLM ready for use with LangChain.
    """

    llm = ChatGoogleGenerativeAI(
        google_api_key=api_key,
        model='gemini-pro',
        temperature=temperature,
        top_p=top_p,
        convert_system_message_to_human=True,
    )

    return llm


##Memory

In [ ]:
def create_memory():
    """
    Creates a ConversationBufferMemory that stores the full chat history
    (no summarization, all messages kept as-is).
    Suitable for models with large context windows.
    """
    memory = ConversationBufferMemory(
        return_messages=True,
        memory_key='chat_history',
        output_key="answer",
        input_key="question",
    )
    return memory


## Prompt template